In [46]:
from tqdm import tqdm

In [47]:
#setting up the detector
from feat import Detector

detector = Detector()
detector

feat.detector.Detector(face_model=retinaface, landmark_model=mobilefacenet, au_model=xgb, emotion_model=resmasknet, facepose_model=img2pose)

In [40]:
def sample(value):
    if(value%2 == 0):
        return 1
    else:
        return 0

In [72]:
# Importing our Utilities functions:
from utils import CautDataloaderRegular
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
    
def RFR_Model(vid_path):

    X_y_data = CautDataloaderRegular.get_X_y_TrainTest(csv_path="C:\\Work\\606Capstone\\Video_chunks\\CSV\\",
                                                       data_dir="C:\\Work\\606Capstone\\Video_chunks\\Excel\\",
                                                       data_mode="OpenFace",
                                                       approach_type="average",  # sequential",
                                                       verbose=True)
    
    X_train, y_train = X_y_data[0], X_y_data[1]
    X_test, y_test = X_y_data[2], X_y_data[3]
    
    #get the dataframe for the test video
    test_video_path = vid_path
    video_prediction = detector.detect_video(test_video_path, skip_frames=24)
    vid_mean = video_prediction.mean()
    vid_mean_df = vid_mean.to_frame()
    vid_mean_df = vid_mean_df.transpose()
    vid_mean_df = vid_mean_df[['AU01','AU02','AU04','AU05','AU06','AU07','AU09','AU10','AU11','AU12','AU14','AU15','AU17','AU20','AU23','AU24','AU25','AU26','AU28','AU43','anger','disgust','fear','happiness','sadness','surprise','neutral']]
    
    # Setup model:
    #fitting and evaluating
    rf = RandomForestClassifier(n_estimators=120)
    
    # fit the model:
    rf.fit(X_train, y_train)
    
    # predict on test data:
    y_pred = rf.predict(vid_mean_df)
    return y_pred[0]


In [ ]:
### code to upload video
import os
import dash
from dash import html, dcc, Input, Output, State

app = dash.Dash()

assets_folder = os.path.join(app.get_asset_url(''), '')
items = os.listdir('assets')

item_list = html.Ul([html.Li(item) for item in items])

app.layout = html.Div([
    #html.H1('Items in the assets folder:'),
    #item_list,
    html.Div(id='video-display', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='video-dropdown',
        options=[{'label': item, 'value': item} for item in items],
        placeholder="Select a video"
    ),
    html.Div([
        html.Button('Display Video', id='video-button')
    ], style={'textAlign': 'center', 'font-size': '24px'})
        
])

@app.callback(
    Output('video-display', 'children'),
    Input('video-button', 'n_clicks'),
    State('video-dropdown', 'value')
)
def display_video(n_clicks, value):
    if n_clicks and value:
        return html.Video(
            src=f'{assets_folder}{value}',
            autoPlay=True,
            controls=True,
            height='400px'
        )

if __name__ == '__main__':
    app.run_server(debug=False)

In [77]:
#### final code

import os
import dash
from dash import dcc, html, Input, Output, State

# Define the asset folder
ASSET_FOLDER = "assets"

# Create the Dash app
#app = dash.Dash(__name__)

#css
external_stylesheets = [
        'https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__,external_stylesheets=external_stylesheets,
                    assets_external_path='./assets/')
    
# Define the layout of the app
app.layout = html.Div([
    html.H1("DECEPTION DETECTION", style={'textAlign': 'center', 'font-size': '24px'}),
    dcc.Dropdown(id='file-list'),
    html.Div([
        html.Div([
            html.Video(id='video-player', controls=True)
        ], style={'margin': '0 auto', 'width': '30%'})
    ], id='video-div', style={'display': 'none'}),
    html.Div([
        html.Button('DETECT', id='play-button')
    ], style={'textAlign': 'center', 'font-size': '24px'}),
    html.Div(id='output', style={'textAlign': 'center', 'font-size': '24px'})
])

# Define the callback to list the files in the asset folder
@app.callback(
    Output('file-list', 'options'),
    [Input('file-list', 'contents')])
def update_file_list(contents):
    if contents:
        # Save the uploaded file to the asset folder
        filename = contents[0]['filename']
        data = contents[0]['contents']
        with open(os.path.join(ASSET_FOLDER, filename), 'wb') as f:
            f.write(data)
    
    # List the files in the asset folder
    file_list = os.listdir(ASSET_FOLDER)
    options = [{'label': f, 'value': f} for f in file_list]
    return options

# Define the callback to display the selected video
@app.callback(
    Output('video-player', 'src'),
    [Input('file-list', 'value')])
def update_video_src(value):
    if value:
        # Set the source of the video player to the selected file
        src = f"/{ASSET_FOLDER}/{value}"
        return src
    else:
        return ""
@app.callback(
    Output('video-div', 'style'),
    [Input('file-list', 'value')])
def update_video_src(value):
    if value:
        return {"display" : "inline-block"}
    else:
        return {"display" : "none"}

# Define the callback to play the video
@app.callback(
    Output('output', 'children'),
    [Input('play-button', 'n_clicks'),
    Input('video-player', 'src')])
def play_video(n_clicks, src):
    if n_clicks is None:
        return f""
    else:
        v = RFR_Model(src[1:])
        if v:
            return f"The person is lying."
        else:
            return f"The person is saying the truth."
    return src

# Reset n_clicks when a different video is selected
@app.callback(
    Output('play-button', 'n_clicks'),
    [Input('file-list', 'value')]
)
def reset_n_clicks(value):
    return None

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
127.0.0.1 - - [17/Apr/2023 19:08:09] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2023 19:08:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 19:08:09] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2023 19:08:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 19:08:09] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2023 19:08:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 19:08:09] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2023 19:08:09] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Apr/2023 19:08:09] "POST /_


Selected csv_path: C:\Work\606Capstone\Video_chunks\CSV\
Processed 100 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 0
Processed 200 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 0
Video failed to be processed by OpenFace. Videoname=trial_truth_025_007.
Processed 300 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 0
Video failed to be processed by OpenFace. Videoname=trial_lie_053_007.
Processed 400 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 1
Processed 500 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 0
Casting collected data to .npy array type...
Data is collected. Returning X and y data.
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


Processed 100 / 223
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 1
Video failed to be processed by OpenFace. Videoname=trial_lie_053_002.
Video failed to be processed by OpenFace. Videoname=trial_truth_025_008.
Video failed t

...

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.27s/it]
C:\Users\hinal\AppData\Local\Temp/ipykernel_21060/366935298.py:31: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

127.0.0.1 - - [17/Apr/2023 19:09:49] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2023 19:09:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 19:10:07] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2023 19:10:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 19:10:08] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2023 19:10:08] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 19:10:08] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2023 19:10:


Selected csv_path: C:\Work\606Capstone\Video_chunks\CSV\
Processed 100 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 0
Processed 200 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 0
Video failed to be processed by OpenFace. Videoname=trial_truth_025_007.
Processed 300 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 0
Video failed to be processed by OpenFace. Videoname=trial_lie_053_007.
Processed 400 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 1
Processed 500 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 0
Casting collected data to .npy array type...
Data is collected. Returning X and y data.
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


Processed 100 / 223
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 1
Video failed to be processed by OpenFace. Videoname=trial_lie_053_002.
Video failed to be processed by OpenFace. Videoname=trial_truth_025_008.
Video failed t

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]127.0.0.1 - - [17/Apr/2023 19:10:23] "GET /_favicon.ico?v=2.6.2 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2023 19:10:23] "GET /_favicon.ico?v=2.6.2 HTTP/1.1" 200 -
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.62s/it]
C:\Users\hinal\AppData\Local\Temp/ipykernel_21060/366935298.py:31: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

127.0.0.1 - - [17/Apr/2023 19:10:27] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2023 19:10:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2023 19:10:31] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2023 


Selected csv_path: C:\Work\606Capstone\Video_chunks\CSV\
Processed 100 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 0
Processed 200 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 0
Video failed to be processed by OpenFace. Videoname=trial_truth_025_007.
Processed 300 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 0
Video failed to be processed by OpenFace. Videoname=trial_lie_053_007.
Processed 400 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 1
Processed 500 / 520
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 0
Casting collected data to .npy array type...
Data is collected. Returning X and y data.
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


Processed 100 / 223
  - Sample shape & label:
    - X_data: (27,)
    - y_data: 1
Video failed to be processed by OpenFace. Videoname=trial_lie_053_002.
Video failed to be processed by OpenFace. Videoname=trial_truth_025_008.
Video failed t

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:13<00:00,  2.26s/it]
C:\Users\hinal\AppData\Local\Temp/ipykernel_21060/366935298.py:31: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

127.0.0.1 - - [17/Apr/2023 19:10:50] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Apr/2023 19:10:50] "POST /_dash-update-component HTTP/1.1" 200 -
